# Relaxed Search(제대로) 가미한 코드

아는 선에서 일단 건드려봤고 1차적인 결과를 올림<br>
알고리즘 돌리는 코드들은 동환이형이 쓴 그대로 썼고,<br>
알고리즘에 따른 셀 분리, relaxed search, best model 등을 구하기만 함. <br>
한번 쭉 봐주고 갖고 놀아보고 이것저것 만져보면 뭐가 뭔지 대충 느낌 올거임.<br>
아님 말구<br>
<strong>plot 은 아직 하나도 안 넣음(ex. 동환이형의 시간축 x 그래프). data visualization 에 필수적인 것이니 꼭 넣어야함</strong>
<br><br>

### 5월 23일 업데이트

<strong> 이 코드 예상 실행 시간... 적으면 2분... 많으면 5분?(데스크탑 기준...)</strong><br>
데이터 양에 따라 차이가 남.


In [1]:
# 최적의 패러미터 가진 모델을 몇번 돌릴지 값.
test_times=10

In [2]:
#데이터 파일 업로드
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import scipy.stats
import random
import warnings
np.set_printoptions(precision=4,suppress=True)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)
plt.rcParams["font.family"]='Malgun Gothic'

아래 셀에서 데이터 수정. <strong> 앞에 Data/ 있어야 제대로 됨</strong><br>
원하는 데이터 이름만 집어넣으면 뒤에는 알아서 될거임

In [3]:
dataname = "ilsan_3years_all.csv"

In [4]:
#데이터 확인 및 필요 없는 열 삭제
pd.set_option('display.max_columns', 500)
df=pd.read_csv(dataname)
#print(df.head())

df.drop(['시군구'],axis=1,inplace=True)
df.drop(['단지명'],axis=1,inplace=True)
df.drop(['건축년도'],axis=1,inplace=True)
# df.drop(['마트최소거리'],axis=1,inplace=True)
# df.drop(['지하철도보거리'],axis=1,inplace=True)
# df.drop(['용적률'],axis=1,inplace=True)
# df.drop(['대형병원최소거리'],axis=1,inplace=True)
#df.drop(['층'],axis=1,inplace=True)
# df.drop(['건폐율'],axis=1,inplace=True)
#df.drop(['총세대수'],axis=1,inplace=True)
# df.drop(['규모별월세가격지수'],axis=1,inplace=True)
# df.drop(['규모별수익률'],axis=1,inplace=True)
# df.drop(['동별인구수'],axis=1,inplace=True)
#print(df.head())

#데이터 형태 실수로 변환
#면적 거래금액 분리버전
Y = df['거래금액'].values
Y = Y.astype('float')
X = df.drop(labels=['거래금액'],axis=1)
X = X.astype('float')

# 단위면적당 거래금액 버전
# df.drop(['전용면적'],axis=1,inplace=True)
# df.drop(['거래금액'],axis=1,inplace=True)
# Y = df['거래금액2'].values
# Y = Y.astype('float')
# X = df.drop(labels=['거래금액2'],axis=1)
# X = X.astype('float')

In [5]:
test_size = [0.1,0.15,0.2,0.25,0.3]
ts_len = len(test_size)

### 1. Random Forest

n_estimators = number of trees in the foreset<br>
max_features = max number of features considered for splitting a node<br>
max_depth = max number of levels in each decision tree<br>
min_samples_split = min number of data points placed in a node before the node is split<br>
min_samples_leaf = min number of data points allowed in a leaf node<br>
bootstrap = method for sampling data points (with or without replacement)

In [6]:
Tree_nums = [5,10,50,100,250,500,750]
Var_nums = [1,3,5,7]

# Random Forest Relaxed Search
x = len(Tree_nums)
y = len(Var_nums)
count=0

# best_RF_RMSE_diff=100
# best_RF_RMSE_train=0
# best_RF_RMSE_test=0
# best_RF_MAPE_train=0
# best_RF_MAPE_test=0
RF_RMSE_train_list =[]
RF_RMSE_test_list =[]
RF_MAPE_train_list =[]
RF_MAPE_test_list =[]
RF_par_diff_list=[]
RF_par_tsize_list=[]
RF_par_trees_list=[]
RF_par_var_list=[]
RF_min_vars_tsize_list=[]
RF_min_vars_trees_list=[]
RF_min_vars_var_list=[]   

print("----progress...----")

for ts in test_size:
    for a in Tree_nums:
        for b in Var_nums:
            random.seed(a*10)
            #훈련셋, 시험셋 분류 
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)
        
            #랜덤포레스트
            RF_model = RandomForestRegressor(n_estimators=a, max_depth=b, max_features=9)
            RF_model.fit(X_train, Y_train)
            
            #예측값 생성
            RF_prediction_test = RF_model.predict(X_test)
            RF_prediction_test = RF_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
            RF_train_test = RF_model.predict(X_train)
            RF_train_test = RF_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
            RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
            RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
            RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
            RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)
            
            # 값들을 위에 지정한 list 에다 다 저장
            RF_RMSE_train_list.append(RF_RMSE_train)
            RF_RMSE_test_list.append(RF_RMSE_test)
            RF_MAPE_train_list.append(RF_MAPE_train)
            RF_MAPE_test_list.append(RF_MAPE_test)
            
            RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)
            
            RF_par_diff_list.append(RF_RMSE_diff)
            RF_par_tsize_list.append(ts)
            RF_par_trees_list.append(a)
            RF_par_var_list.append(b)
            
#             if best_RF_RMSE_diff>RF_RMSE_diff:
#                 best_RF_RMSE_diff = RF_RMSE_diff
#                 best_RF_RMSE_train = RF_RMSE_train
#                 best_RF_RMSE_test = RF_RMSE_test
#                 best_RF_MAPE_train = RF_MAPE_train
#                 best_RF_MAPE_test = RF_MAPE_test
#                 RF_C = count+1
#                 RF_TS = ts
#                 RF_A = a
#                 RF_B = b
         
                
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                  .format(progress,count,ts,a,b,RF_RMSE_diff,RF_RMSE_train,RF_RMSE_test,RF_MAPE_train,RF_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  Tree_num: {} , Var_num: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(RF_C,RF_TS,RF_A,RF_B,best_RF_RMSE_train,best_RF_RMSE_test,best_RF_RMSE_diff,best_RF_MAPE_train,best_RF_MAPE_test))
                


for i in range(5):
    val = min(RF_par_diff_list)
    index = RF_par_diff_list.index(val)
    RF_min_vars_tsize_list.append(RF_par_tsize_list.pop(index))
    RF_min_vars_trees_list.append(RF_par_trees_list.pop(index))
    RF_min_vars_var_list.append(RF_par_var_list.pop(index))


----progress...----
 0.71% , case:1 , test_size: 0.1 , Tree_num: 5 , Var_num: 1 , difference(%) : 10.49
 ---  train RMSE : 7512 , test RMSE : 6799
---  train MAPE : 30.72 , test MAPE : 31.45
 1.43% , case:2 , test_size: 0.1 , Tree_num: 5 , Var_num: 3 , difference(%) : 2.37
 ---  train RMSE : 4282 , test RMSE : 4386
---  train MAPE : 16.25 , test MAPE : 16.02
 2.14% , case:3 , test_size: 0.1 , Tree_num: 5 , Var_num: 5 , difference(%) : 3.83
 ---  train RMSE : 2913 , test RMSE : 3029
---  train MAPE : 10.04 , test MAPE : 9.98
 2.86% , case:4 , test_size: 0.1 , Tree_num: 5 , Var_num: 7 , difference(%) : 19.07
 ---  train RMSE : 2228 , test RMSE : 2753
---  train MAPE : 7.57 , test MAPE : 8.3
 3.57% , case:5 , test_size: 0.1 , Tree_num: 10 , Var_num: 1 , difference(%) : 18.11
 ---  train RMSE : 7944 , test RMSE : 6726
---  train MAPE : 25.26 , test MAPE : 24.96
 4.29% , case:6 , test_size: 0.1 , Tree_num: 10 , Var_num: 3 , difference(%) : 3.80
 ---  train RMSE : 4071 , test RMSE : 3922
---

In [7]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("test_size : {} , Tree_num : {}, Var_num : {}".format(RF_min_vars_tsize_list[i],RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))

print("...testing...")

progress=0
count=0
RF_Best_RMSE_train_list=[]
RF_Best_RMSE_test_list=[]
RF_Best_MAPE_train_list=[]
RF_Best_MAPE_test_list=[]
RF_Best_RMSE_diff_list=[]

RF_Best_total_data=[]

for i in range(5):
    temp_RF_Best_RMSE_train_list=[]
    temp_RF_Best_RMSE_test_list=[]
    temp_RF_Best_MAPE_train_list=[]
    temp_RF_Best_MAPE_test_list=[]
    temp_RF_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = RF_min_vars_tsize_list[i]
        a = RF_min_vars_trees_list[i]
        b = RF_min_vars_var_list[i]
        random.seed((k+1)*10)

        #훈련셋, 시험셋 분류 
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        #랜덤포레스트
        RF_Best_model = RandomForestRegressor(n_estimators=a, max_depth=b, max_features=9)
        RF_Best_model.fit(X_train, Y_train)

            #예측값 생성
        RF_Best_prediction_test = RF_Best_model.predict(X_test)
        RF_Best_prediction_test = RF_Best_prediction_test.astype('int')

            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
        RF_Best_train_test = RF_Best_model.predict(X_train)
        RF_Best_train_test = RF_Best_train_test.astype('int')

            #훈련셋 RMSE & MAPE
        RF_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_Best_train_test)).astype('int')
        RF_Best_MAPE_train = round(np.mean(np.abs((Y_train - RF_Best_train_test) / Y_train)) * 100 ,2)

            #시험셋 RMSE & MAPE
        RF_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_Best_prediction_test)).astype('int')
        RF_Best_MAPE_test = round(np.mean(np.abs((Y_test - RF_Best_prediction_test) / Y_test)) * 100 ,2)

        RF_Best_RMSE_diff = np.abs((RF_Best_RMSE_test-RF_Best_RMSE_train)/RF_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_RF_Best_RMSE_train_list.append(RF_Best_RMSE_train)
        temp_RF_Best_RMSE_test_list.append(RF_Best_RMSE_test)
        temp_RF_Best_MAPE_train_list.append(RF_Best_MAPE_train)
        temp_RF_Best_MAPE_test_list.append(RF_Best_MAPE_test)
        temp_RF_Best_RMSE_diff_list.append(RF_Best_RMSE_diff)
        
        
        count+=1
        progress = round(count/(10*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,a,b,RF_Best_RMSE_diff,RF_Best_RMSE_train,RF_Best_RMSE_test,RF_Best_MAPE_train,RF_Best_MAPE_test))
    
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_diff_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_diff_list))
    
#     RF_Best_RMSE_train_list.append(temp_RF_Best_RMSE_train_list)
#     RF_Best_RMSE_test_list.append(temp_RF_Best_RMSE_test_list)
#     RF_Best_MAPE_train_list.append(temp_RF_Best_MAPE_train_list)
#     RF_Best_MAPE_test_list.append(temp_RF_Best_MAPE_test_list)
#     RF_Best_RMSE_diff_list.append(temp_RF_Best_RMSE_diff_list)
    
    
# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} , Tree_num: {} , Var_num: {} ,   ---  MAPE : {}".format(RF_Best_C,RF_Best_TS,RF_Best_A,RF_Best_B,RF_Best_min))

PARAMETERS FOR BEST RF...
test_size : 0.25 , Tree_num : 750, Var_num : 1
test_size : 0.25 , Tree_num : 750, Var_num : 3
test_size : 0.25 , Tree_num : 750, Var_num : 5
test_size : 0.25 , Tree_num : 750, Var_num : 7
test_size : 0.3 , Tree_num : 5, Var_num : 1
...testing...
 2.00% , case:1 , test_size: 0.25 , Tree_num: 750 , Var_num: 1 , difference(%) : 3.60
 ---  train RMSE : 7287 , test RMSE : 7034
---  train MAPE : 26.69 , test MAPE : 25.93
 4.00% , case:2 , test_size: 0.25 , Tree_num: 750 , Var_num: 1 , difference(%) : 7.74
 ---  train RMSE : 7054 , test RMSE : 7646
---  train MAPE : 26.79 , test MAPE : 27.52
 6.00% , case:3 , test_size: 0.25 , Tree_num: 750 , Var_num: 1 , difference(%) : 0.18
 ---  train RMSE : 7230 , test RMSE : 7243
---  train MAPE : 26.56 , test MAPE : 26.42
 8.00% , case:4 , test_size: 0.25 , Tree_num: 750 , Var_num: 1 , difference(%) : 4.20
 ---  train RMSE : 7157 , test RMSE : 7471
---  train MAPE : 26.51 , test MAPE : 26.8
 10.00% , case:5 , test_size: 0.25 , 

In [8]:
rf1=[]
rf2=[]
rf3=[]
rf4=[]
rf5=[]

for i in range(50):
    k=i//10
    if k==0:
        rf1.append(RF_Best_total_data[i])
    if k==1:
        rf2.append(RF_Best_total_data[i])
    if k==2:
        rf3.append(RF_Best_total_data[i])
    if k==3:
        rf4.append(RF_Best_total_data[i])
    if k==4:
        rf5.append(RF_Best_total_data[i])
    
RF_s1 = pd.Series(rf1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

RF_s2 = pd.Series(rf2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
RF_s3 = pd.Series(rf3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
RF_s4 = pd.Series(rf4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
RF_s5 = pd.Series(rf5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

RF_best_total_dataframe = pd.concat([RF_s1,RF_s2,RF_s3,RF_s4,RF_s5],axis=1)
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,7240.900000,4012.900000,2725.400000,1925.000000,7436.000000
RMSE test Mean,7133.500000,4090.300000,3082.300000,2802.700000,7312.700000
RMSE train Stdev,120.588101,49.247233,32.028737,91.856410,282.209497
RMSE test Stdev,404.628533,124.288415,130.105380,433.140635,533.123447
MAPE train Mean,26.701000,14.623000,9.852000,6.920000,27.043000
MAPE test Mean,26.374000,14.724000,10.321000,7.871000,26.887000
MAPE train Stdev,0.178519,0.151858,0.103131,0.093595,1.727119
MAPE test Stdev,0.749482,0.669525,0.347432,0.490417,1.446935
RMSE diff(%) Mean,6.022603,3.805008,11.388032,29.310225,6.188169
RMSE diff(%) Stdev,5.210381,2.211401,4.597460,13.125633,3.621313


### 2. Support Vector Machine parameters

In [9]:
gamma_nums = [0.00015,0.001,0.01,0.1]
C_values= [500,1000,5000,10000,50000]
x = len(gamma_nums)
y = len(C_values)
count=0
# Random Forest Relaxed Search
count=0

# best_SVR_RMSE_diff=100
# best_SVR_RMSE_train=0
# best_SVR_RMSE_test=0
# best_SVR_MAPE_train=0
# best_SVR_MAPE_test=0
SVR_RMSE_train_list =[]
SVR_RMSE_test_list =[]
SVR_MAPE_train_list =[]
SVR_MAPE_test_list =[]
SVR_par_diff_list=[]
SVR_par_tsize_list=[]
SVR_par_gamma_list=[]
SVR_par_cval_list=[]
SVR_min_vars_tsize_list=[]
SVR_min_vars_gamma_list=[]
SVR_min_vars_cval_list=[]   

print("----progress...----")

for ts in test_size:
    for a in gamma_nums:
        for b in C_values:
            random.seed(a*10)
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

            SVR_model = svm.SVR(kernel='rbf',gamma=a,C=b)
            SVR_model.fit(X_train, Y_train)
            
            #예측값 생성
            SVR_prediction_test = SVR_model.predict(X_test)
            SVR_prediction_test = SVR_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
            SVR_train_test = SVR_model.predict(X_train)
            SVR_train_test = SVR_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
            SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
            SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
            SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
            SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
            # 값들을 위에 지정한 list 에다 다 저장
            SVR_RMSE_train_list.append(SVR_RMSE_train)
            SVR_RMSE_test_list.append(SVR_RMSE_test)
            SVR_MAPE_train_list.append(SVR_MAPE_train)
            SVR_MAPE_test_list.append(SVR_MAPE_test)
            
            SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
            SVR_par_diff_list.append(SVR_RMSE_diff)
            SVR_par_tsize_list.append(ts)
            SVR_par_gamma_list.append(a)
            SVR_par_cval_list.append(b)
            
#             if best_SVR_RMSE_diff>SVR_RMSE_diff:
#                 best_SVR_RMSE_diff = SVR_RMSE_diff
#                 best_SVR_RMSE_train = SVR_RMSE_train
#                 best_SVR_RMSE_test = SVR_RMSE_test
#                 best_SVR_MAPE_train = SVR_MAPE_train
#                 best_SVR_MAPE_test = SVR_MAPE_test
#                 SVR_C = count+1
#                 SVR_TS = ts
#                 SVR_A = a
#                 SVR_B = b
         
                
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                  .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  Gamma_size: {} ,  C_values: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(SVR_C,SVR_TS,SVR_A,SVR_B,best_SVR_RMSE_train,best_SVR_RMSE_test,best_SVR_RMSE_diff,best_SVR_MAPE_train,best_SVR_MAPE_test))
                


for i in range(5):
    val = min(SVR_par_diff_list)
    index = SVR_par_diff_list.index(val)
    SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
    SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
    SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


----progress...----
 1.00% , case:1 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 500 , difference(%) : 21.50
 ---  train RMSE : 7658 , test RMSE : 9755
---  train MAPE : 13.36 , test MAPE : 13.33
 2.00% , case:2 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 1000 , difference(%) : 17.41
 ---  train RMSE : 6278 , test RMSE : 5347
---  train MAPE : 9.7 , test MAPE : 10.62
 3.00% , case:3 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 5000 , difference(%) : 2.04
 ---  train RMSE : 3944 , test RMSE : 4026
---  train MAPE : 6.74 , test MAPE : 6.98
 4.00% , case:4 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 10000 , difference(%) : 13.15
 ---  train RMSE : 3401 , test RMSE : 3916
---  train MAPE : 5.96 , test MAPE : 7.23
 5.00% , case:5 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 50000 , difference(%) : 21.07
 ---  train RMSE : 2679 , test RMSE : 3394
---  train MAPE : 4.92 , test MAPE : 6.34
 6.00% , case:6 , test_size: 0.1 , Gamma_size: 0.001 ,  C_values: 50

In [10]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

print("\n\n...testing...\n")
progress=0
count=0
SVR_Best_RMSE_train_list=[]
SVR_Best_RMSE_test_list=[]
SVR_Best_MAPE_train_list=[]
SVR_Best_MAPE_test_list=[]
SVR_Best_RMSE_diff_list=[]

SVR_Best_total_data=[]

for i in range(5):
    temp_SVR_Best_RMSE_train_list=[]
    temp_SVR_Best_RMSE_test_list=[]
    temp_SVR_Best_MAPE_train_list=[]
    temp_SVR_Best_MAPE_test_list=[]
    temp_SVR_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = SVR_min_vars_tsize_list[i]
        a = SVR_min_vars_gamma_list[i]
        b = SVR_min_vars_cval_list[i]
        random.seed((k+1)*10)

        #훈련셋, 시험셋 분류 
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        SVR_Best_model = svm.SVR(kernel='rbf',gamma=a,C=b)
        SVR_Best_model.fit(X_train, Y_train)
            
            #예측값 생성
        SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
        SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
        SVR_Best_train_test = SVR_Best_model.predict(X_train)
        SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
        SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
        SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
        SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
        SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
        SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
        temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
        temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
        temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
        temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

    #     if SVR_Best_min>SVR_Best_MAPE_test:
    #         SVR_Best_min = SVR_Best_MAPE_test
    #         SVR_Best_TS = ts
    #         SVR_Best_A = a
    #         SVR_Best_B = b
    #         SVR_Best_C = count+1

        count+=1
        progress = round(count/(test_times*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))

    
    


PARAMETERS FOR BEST SVR...
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 500
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 1000
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 5000
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 10000
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 50000


...testing...

 2.00% , case:1 , test_size: 0.25 , Tree_num: 0.001 , Var_num: 500 , difference(%) : 4.35
 ---  train RMSE : 10383 , test RMSE : 10855
---  train MAPE : 17.59 , test MAPE : 17.79
 4.00% , case:2 , test_size: 0.25 , Tree_num: 0.001 , Var_num: 500 , difference(%) : 3.00
 ---  train RMSE : 10437 , test RMSE : 10760
---  train MAPE : 17.67 , test MAPE : 19.17
 6.00% , case:3 , test_size: 0.25 , Tree_num: 0.001 , Var_num: 500 , difference(%) : 3.16
 ---  train RMSE : 10302 , test RMSE : 10638
---  train MAPE : 17.97 , test MAPE : 18.13
 8.00% , case:4 , test_size: 0.25 , Tree_num: 0.001 , Var_num: 500 , difference(%) : 5.50
 ---  train RMSE : 10356 , test RMSE : 10959
---  tra

In [11]:
SVR1=[]
SVR2=[]
SVR3=[]
SVR4=[]
SVR5=[]

for i in range(50):
    k=i//10
    if k==0:
        SVR1.append(SVR_Best_total_data[i])
    if k==1:
        SVR2.append(SVR_Best_total_data[i])
    if k==2:
        SVR3.append(SVR_Best_total_data[i])
    if k==3:
        SVR4.append(SVR_Best_total_data[i])
    if k==4:
        SVR5.append(SVR_Best_total_data[i])

    
SVR_s1 = pd.Series(SVR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

SVR_s2 = pd.Series(SVR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
SVR_s3 = pd.Series(SVR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
SVR_s4 = pd.Series(SVR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
SVR_s5 = pd.Series(SVR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

SVR_best_total_dataframe_rbf = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
print("SVR best 5 models")
SVR_best_total_dataframe_rbf

SVR best 5 models


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,10392.900000,7554.600000,4268.900000,3384.000000,2205.400000
RMSE test Mean,10460.400000,8150.600000,4763.400000,4080.200000,3200.300000
RMSE train Stdev,99.491155,142.867211,81.416767,143.343643,58.912138
RMSE test Stdev,424.906625,579.078786,409.680412,426.055818,280.379760
MAPE train Mean,17.776000,12.020000,5.815000,4.655000,3.264000
MAPE test Mean,18.521000,13.348000,8.074000,7.309000,6.483000
MAPE train Stdev,0.221414,0.200000,0.078645,0.121840,0.057827
MAPE test Stdev,0.702374,0.858904,0.493360,0.487164,0.343600
RMSE diff(%) Mean,4.187966,7.237068,9.695551,16.616342,30.561188
RMSE diff(%) Stdev,2.235973,6.765251,8.095203,9.571676,6.388569


### EXTRA... SVM LINEAR KERNEL

In [12]:
# gamma_nums = [0.001,0.01,0.1,1]
# C_values= [10,100,1000,10000]
# x = len(gamma_nums)
# y = len(C_values)
# count=0
# # Random Forest Relaxed Search
# count=0

# # best_SVR_RMSE_diff=100
# # best_SVR_RMSE_train=0
# # best_SVR_RMSE_test=0
# # best_SVR_MAPE_train=0
# # best_SVR_MAPE_test=0
# SVR_RMSE_train_list =[]
# SVR_RMSE_test_list =[]
# SVR_MAPE_train_list =[]
# SVR_MAPE_test_list =[]
# SVR_par_diff_list=[]
# SVR_par_tsize_list=[]
# SVR_par_gamma_list=[]
# SVR_par_cval_list=[]
# SVR_min_vars_tsize_list=[]
# SVR_min_vars_gamma_list=[]
# SVR_min_vars_cval_list=[]   

# print("----progress...----")

# for ts in test_size:
#     for a in gamma_nums:
#         for b in C_values:
#             random.seed(a*10)
#             X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#             SVR_model = svm.SVR(kernel='linear',gamma=a,C=b)
#             SVR_model.fit(X_train, Y_train)
            
#             #예측값 생성
#             SVR_prediction_test = SVR_model.predict(X_test)
#             SVR_prediction_test = SVR_prediction_test.astype('int')
            
#             #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
#             SVR_train_test = SVR_model.predict(X_train)
#             SVR_train_test = SVR_train_test.astype('int')
            
#             #훈련셋 RMSE & MAPE
#             SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
#             SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
#             #시험셋 RMSE & MAPE
#             SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
#             SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
#             # 값들을 위에 지정한 list 에다 다 저장
#             SVR_RMSE_train_list.append(SVR_RMSE_train)
#             SVR_RMSE_test_list.append(SVR_RMSE_test)
#             SVR_MAPE_train_list.append(SVR_MAPE_train)
#             SVR_MAPE_test_list.append(SVR_MAPE_test)
            
#             SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
#             SVR_par_diff_list.append(SVR_RMSE_diff)
#             SVR_par_tsize_list.append(ts)
#             SVR_par_gamma_list.append(a)
#             SVR_par_cval_list.append(b)
            
# #             if best_SVR_RMSE_diff>SVR_RMSE_diff:
# #                 best_SVR_RMSE_diff = SVR_RMSE_diff
# #                 best_SVR_RMSE_train = SVR_RMSE_train
# #                 best_SVR_RMSE_test = SVR_RMSE_test
# #                 best_SVR_MAPE_train = SVR_MAPE_train
# #                 best_SVR_MAPE_test = SVR_MAPE_test
# #                 SVR_C = count+1
# #                 SVR_TS = ts
# #                 SVR_A = a
# #                 SVR_B = b
         
                
#             count+=1
#             progress = round(count/(x*y*ts_len)*100,2)
            
#             print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            

# # print("\n\n")
# # print("Best Case")
# # print("case:{} , test_size: {} ,  Gamma_size: {} ,  C_values: {} ,  \n"
# #       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
# #       "---  train MAPE : {} , test MAPE : {}"
# #       .format(SVR_C,SVR_TS,SVR_A,SVR_B,best_SVR_RMSE_train,best_SVR_RMSE_test,best_SVR_RMSE_diff,best_SVR_MAPE_train,best_SVR_MAPE_test))
                


# for i in range(5):
#     val = min(SVR_par_diff_list)
#     index = SVR_par_diff_list.index(val)
#     SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
#     SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
#     SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


In [13]:
# print("PARAMETERS FOR BEST SVR...")
# for i in range(5):
#     print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

# print("\n\n...testing...\n")
# test_times=5
# progress=0
# count=0
# SVR_Best_RMSE_train_list=[]
# SVR_Best_RMSE_test_list=[]
# SVR_Best_MAPE_train_list=[]
# SVR_Best_MAPE_test_list=[]
# SVR_Best_RMSE_diff_list=[]

# SVR_Best_total_data=[]

# for i in range(5):
#     temp_SVR_Best_RMSE_train_list=[]
#     temp_SVR_Best_RMSE_test_list=[]
#     temp_SVR_Best_MAPE_train_list=[]
#     temp_SVR_Best_MAPE_test_list=[]
#     temp_SVR_Best_RMSE_diff_list=[]
#     for k in range(test_times):
#         ts = SVR_min_vars_tsize_list[i]
#         a = SVR_min_vars_gamma_list[i]
#         b = SVR_min_vars_cval_list[i]
#         random.seed((k+1)*10)

#         #훈련셋, 시험셋 분류 
#         X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#         SVR_Best_model = svm.SVR(kernel='linear',gamma=a,C=b)
#         SVR_Best_model.fit(X_train, Y_train)
            
#             #예측값 생성
#         SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
#         SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
#             #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
#         SVR_Best_train_test = SVR_Best_model.predict(X_train)
#         SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
#             #훈련셋 RMSE & MAPE
#         SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
#         SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
#             #시험셋 RMSE & MAPE
#         SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
#         SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
#         SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
#         # 값들을 위에 지정한 list 에다 다 저장
#         temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
#         temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
#         temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
#         temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
#         temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

#     #     if SVR_Best_min>SVR_Best_MAPE_test:
#     #         SVR_Best_min = SVR_Best_MAPE_test
#     #         SVR_Best_TS = ts
#     #         SVR_Best_A = a
#     #         SVR_Best_B = b
#     #         SVR_Best_C = count+1

#         count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))


In [14]:
# SVR1=[]
# SVR2=[]
# SVR3=[]
# SVR4=[]
# SVR5=[]

# for i in range(50):
#     k=i//10
#     if k==0:
#         SVR1.append(SVR_Best_total_data[i])
#     if k==1:
#         SVR2.append(SVR_Best_total_data[i])
#     if k==2:
#         SVR3.append(SVR_Best_total_data[i])
#     if k==3:
#         SVR4.append(SVR_Best_total_data[i])
#     if k==4:
#         SVR5.append(SVR_Best_total_data[i])

    
# SVR_s1 = pd.Series(SVR1,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 1')

# SVR_s2 = pd.Series(SVR2,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 2')
# SVR_s3 = pd.Series(SVR3,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 3')
# SVR_s4 = pd.Series(SVR4,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 4')
# SVR_s5 = pd.Series(SVR5,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 5')

# SVR_best_total_dataframe_linear = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
# print("SVR best 5 models")
# SVR_best_total_dataframe_linear

### 3. Linear Regression ... <br>
what parameters are there here?

In [12]:
times=10
count=0

# best_LR_RMSE_diff=100
# best_LR_RMSE_train=0
# best_LR_RMSE_test=0
# best_LR_MAPE_train=0
# best_LR_MAPE_test=0
LR_RMSE_train_list =[]
LR_RMSE_test_list =[]
LR_MAPE_train_list =[]
LR_MAPE_test_list =[]
LR_par_diff_list=[]
LR_par_tsize_list=[]
LR_min_vars_tsize_list=[] 

print("----progress...----")

for ts in test_size:
    for i in range(times):
        random.seed(a*10)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        LR_model=LinearRegression()
        LR_model.fit(X_train, Y_train)
        LR_prediction_test = LR_model.predict(X_test)
        LR_prediction_test = LR_prediction_test.astype('int')
        LR_train_test = LR_model.predict(X_train)
        LR_train_test = LR_train_test.astype('int')
    
        #훈련셋 RMSE & MAPE
        LR_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_train_test)).astype('int')
        LR_MAPE_train = round(np.mean(np.abs((Y_train - LR_train_test) / Y_train)) * 100 ,2)
            
        #시험셋 RMSE & MAPE
        LR_RMSE_test =np.sqrt(mean_squared_error(Y_test, LR_prediction_test)).astype('int')
        LR_MAPE_test = round(np.mean(np.abs((Y_test - LR_prediction_test) / Y_test)) * 100 ,2)
        
        # 값들을 위에 지정한 list 에다 다 저장
        LR_RMSE_train_list.append(LR_RMSE_train)
        LR_RMSE_test_list.append(LR_RMSE_test)
        LR_MAPE_train_list.append(LR_MAPE_train)
        LR_MAPE_test_list.append(LR_MAPE_test)
            
        LR_RMSE_diff = np.abs((LR_RMSE_test-LR_RMSE_train)/LR_RMSE_test *100)
            
        LR_par_diff_list.append(LR_RMSE_diff)
        LR_par_tsize_list.append(ts)
            
#             if best_LR_RMSE_diff>LR_RMSE_diff:
#                 best_LR_RMSE_diff = LR_RMSE_diff
#                 best_LR_RMSE_train = LR_RMSE_train
#                 best_LR_RMSE_test = LR_RMSE_test
#                 best_LR_MAPE_train = LR_MAPE_train
#                 best_LR_MAPE_test = LR_MAPE_test
#                 LR_C = count+1
#                 LR_TS = ts
#                 LR_A = a
#                 LR_B = b
         
                
        count+=1
        progress = round(count/(ts_len*times)*100,2)
            
        print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
        "---  train RMSE : {} , test RMSE : {}\n"
        "---  train MAPE : {} , test MAPE : {}"
                .format(progress,count,ts,LR_RMSE_diff,LR_RMSE_train,LR_RMSE_test,LR_MAPE_train,LR_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(LR_C,LR_TS,best_LR_RMSE_train,best_LR_RMSE_test,best_LR_RMSE_diff,best_LR_MAPE_train,best_LR_MAPE_test))
                


for i in range(5):
    val = min(LR_par_diff_list)
    index = LR_par_diff_list.index(val)
    LR_min_vars_tsize_list.append(LR_par_tsize_list.pop(index))


----progress...----
 2.00% , case:1 , test_size: 0.1 , difference(%) : 8.34
 ---  train RMSE : 4454 , test RMSE : 4111
---  train MAPE : 13.24 , test MAPE : 13.4
 4.00% , case:2 , test_size: 0.1 , difference(%) : 0.25
 ---  train RMSE : 4424 , test RMSE : 4413
---  train MAPE : 13.08 , test MAPE : 12.37
 6.00% , case:3 , test_size: 0.1 , difference(%) : 20.83
 ---  train RMSE : 4307 , test RMSE : 5440
---  train MAPE : 12.32 , test MAPE : 16.95
 8.00% , case:4 , test_size: 0.1 , difference(%) : 37.63
 ---  train RMSE : 4528 , test RMSE : 3290
---  train MAPE : 13.42 , test MAPE : 12.59
 10.00% , case:5 , test_size: 0.1 , difference(%) : 13.11
 ---  train RMSE : 4469 , test RMSE : 3951
---  train MAPE : 13.19 , test MAPE : 12.32
 12.00% , case:6 , test_size: 0.1 , difference(%) : 9.22
 ---  train RMSE : 4456 , test RMSE : 4080
---  train MAPE : 13.39 , test MAPE : 11.81
 14.00% , case:7 , test_size: 0.1 , difference(%) : 19.68
 ---  train RMSE : 4315 , test RMSE : 5372
---  train MAPE :

In [13]:
print("PARAMETERS FOR BEST LR...")
for i in range(5):
    print("test_size : {} ".format(LR_min_vars_tsize_list[i]))

print("...testing...")

progress=0
count=0
LR_Best_RMSE_train_list=[]
LR_Best_RMSE_test_list=[]
LR_Best_MAPE_train_list=[]
LR_Best_MAPE_test_list=[]
LR_Best_RMSE_diff_list=[]

LR_Best_total_data=[]

for i in range(5):
    temp_LR_Best_RMSE_train_list=[]
    temp_LR_Best_RMSE_test_list=[]
    temp_LR_Best_MAPE_train_list=[]
    temp_LR_Best_MAPE_test_list=[]
    temp_LR_Best_RMSE_diff_list=[]
    
    for k in range(test_times):
        ts = LR_min_vars_tsize_list[i]
        random.seed((k+1)*10)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        LR_Best_model=LinearRegression()
        LR_Best_model.fit(X_train, Y_train)
        LR_Best_prediction_test = LR_Best_model.predict(X_test)
        LR_Best_prediction_test = LR_Best_prediction_test.astype('int')
        LR_Best_train_test = LR_Best_model.predict(X_train)
        LR_Best_train_test = LR_Best_train_test.astype('int')

        #훈련셋 RMSE & MAPE
        LR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_Best_train_test)).astype('int')
        LR_Best_MAPE_train = round(np.mean(np.abs((Y_train - LR_Best_train_test) / Y_train)) * 100 ,2)

        #시험셋 RMSE & MAPE
        LR_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, LR_Best_prediction_test)).astype('int')
        LR_Best_MAPE_test = round(np.mean(np.abs((Y_test - LR_Best_prediction_test) / Y_test)) * 100 ,2)

        LR_Best_RMSE_diff = np.abs((LR_Best_RMSE_test-LR_Best_RMSE_train)/LR_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_LR_Best_RMSE_train_list.append(LR_Best_RMSE_train)
        temp_LR_Best_RMSE_test_list.append(LR_Best_RMSE_test)
        temp_LR_Best_MAPE_train_list.append(LR_Best_MAPE_train)
        temp_LR_Best_MAPE_test_list.append(LR_Best_MAPE_test)
        temp_LR_Best_RMSE_diff_list.append(LR_Best_RMSE_diff)
            

    #     if LR_Best_min>LR_Best_MAPE_test:
    #         LR_Best_min = LR_Best_MAPE_test
    #         LR_Best_TS = ts
    #         LR_Best_A = a
    #         LR_Best_B = b
    #         LR_Best_C = count+1

        count+=1
        progress = round(count/(test_times*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,LR_Best_RMSE_diff,LR_Best_RMSE_train,LR_Best_RMSE_test,LR_Best_MAPE_train,LR_Best_MAPE_test))
    
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_diff_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_diff_list))
    

#     LR_Best_RMSE_train_list.append(temp_LR_Best_RMSE_train_list)
#     LR_Best_RMSE_test_list.append(temp_LR_Best_RMSE_test_list)
#     LR_Best_MAPE_train_list.append(temp_LR_Best_MAPE_train_list)
#     LR_Best_MAPE_test_list.append(temp_LR_Best_MAPE_test_list)
#     LR_Best_RMSE_diff_list.append(temp_LR_Best_RMSE_diff_list)
    
    
# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} , Tree_num: {} , Var_num: {} ,   ---  MAPE : {}".format(LR_Best_C,LR_Best_TS,LR_Best_A,LR_Best_B,LR_Best_min))



PARAMETERS FOR BEST LR...
test_size : 0.1 
test_size : 0.1 
test_size : 0.1 
test_size : 0.1 
test_size : 0.1 
...testing...
 2.00% , case:1 , test_size: 0.1 , difference(%) : 8.14
 ---  train RMSE : 4389 , test RMSE : 4778
---  train MAPE : 12.6 , test MAPE : 16.5
 4.00% , case:2 , test_size: 0.1 , difference(%) : 6.06
 ---  train RMSE : 4405 , test RMSE : 4689
---  train MAPE : 12.45 , test MAPE : 15.88
 6.00% , case:3 , test_size: 0.1 , difference(%) : 8.90
 ---  train RMSE : 4455 , test RMSE : 4091
---  train MAPE : 13.23 , test MAPE : 12.25
 8.00% , case:4 , test_size: 0.1 , difference(%) : 12.94
 ---  train RMSE : 4468 , test RMSE : 3956
---  train MAPE : 13.34 , test MAPE : 12.38
 10.00% , case:5 , test_size: 0.1 , difference(%) : 2.09
 ---  train RMSE : 4412 , test RMSE : 4506
---  train MAPE : 13.21 , test MAPE : 12.34
 12.00% , case:6 , test_size: 0.1 , difference(%) : 2.81
 ---  train RMSE : 4434 , test RMSE : 4313
---  train MAPE : 13.22 , test MAPE : 11.98
 14.00% , case:7

In [14]:


LR1=[]
LR2=[]
LR3=[]
LR4=[]
LR5=[]

for i in range(50):
    k=i//10
    if k==0:
        LR1.append(LR_Best_total_data[i])
    if k==1:
        LR2.append(LR_Best_total_data[i])
    if k==2:
        LR3.append(LR_Best_total_data[i])
    if k==3:
        LR4.append(LR_Best_total_data[i])
    if k==4:
        LR5.append(LR_Best_total_data[i])
    
LR_s1 = pd.Series(LR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

LR_s2 = pd.Series(LR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
LR_s3 = pd.Series(LR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
LR_s4 = pd.Series(LR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
LR_s5 = pd.Series(LR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

LR_best_total_dataframe = pd.concat([LR_s1,LR_s2,LR_s3,LR_s4,LR_s5],axis=1)

LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,4395.900000,4412.400000,4382.600000,4415.100000,4399.000000
RMSE test Mean,4659.400000,4473.900000,4654.800000,4468.400000,4596.400000
RMSE train Stdev,54.277896,54.715994,125.361238,63.181405,60.514461
RMSE test Stdev,481.857074,475.213731,1006.766885,580.909838,542.971491
MAPE train Mean,12.956000,13.131000,12.990000,12.994000,13.073000
MAPE test Mean,13.881000,12.564000,13.859000,13.724000,13.388000
MAPE train Stdev,0.358753,0.303198,0.434810,0.396540,0.321031
MAPE test Stdev,1.850635,1.402257,1.895597,2.088637,1.880882
RMSE diff(%) Mean,9.477170,9.223154,17.799884,12.850203,10.108644
RMSE diff(%) Stdev,6.826210,5.802936,13.146803,6.530424,7.383311


In [25]:
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,2750.900000,13576.500000,7660.600000,4405.700000,2753.100000
RMSE test Mean,3818.500000,13468.500000,7732.300000,5145.000000,3916.500000
RMSE train Stdev,102.243288,257.727472,156.861213,52.176719,114.665993
RMSE test Stdev,102.243288,257.727472,156.861213,52.176719,114.665993
MAPE train Mean,5.784000,30.287000,15.987000,9.257000,5.810000
MAPE test Mean,6.436000,30.898000,16.129000,9.643000,6.368000
MAPE train Stdev,0.045431,0.255893,0.192824,0.043600,0.054406
MAPE test Stdev,0.045431,0.255893,0.192824,0.043600,0.054406
RMSE diff(%) Mean,22.577785,9.833896,12.405086,14.836515,23.218189
RMSE diff(%) Stdev,17.462218,6.440763,11.438193,12.553633,18.826700


In [26]:
SVR_best_total_dataframe_rbf

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,20720.500000,19546.100000,13443.800000,9941.300000,4204.400000
RMSE test Mean,21439.900000,19283.100000,14283.200000,11624.700000,8332.200000
RMSE train Stdev,514.682475,291.162652,626.813178,810.253917,1185.274584
RMSE test Stdev,514.682475,291.162652,626.813178,810.253917,1185.274584
MAPE train Mean,20.678000,16.787000,7.376000,4.293000,1.330000
MAPE test Mean,22.460000,18.765000,11.138000,9.643000,8.063000
MAPE train Stdev,0.163328,0.159878,0.091016,0.061327,0.037947
MAPE test Stdev,0.163328,0.159878,0.091016,0.061327,0.037947
RMSE diff(%) Mean,6.527317,5.452579,13.580089,17.886906,42.049577
RMSE diff(%) Stdev,6.374096,3.833151,8.563250,16.788778,28.929207


In [27]:
LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,7871.100000,7776.300000,7976.000000,7841.000000,7706.700000
RMSE test Mean,7654.100000,7898.400000,7352.900000,7782.700000,8143.700000
RMSE train Stdev,327.304277,404.225445,240.579717,239.564605,323.381524
RMSE test Stdev,327.304277,404.225445,240.579717,239.564605,323.381524
MAPE train Mean,16.439000,16.574000,16.607000,16.490000,16.429000
MAPE test Mean,16.595000,16.448000,16.559000,16.474000,16.460000
MAPE train Stdev,0.154690,0.183478,0.190686,0.159750,0.224920
MAPE test Stdev,0.154690,0.183478,0.190686,0.159750,0.224920
RMSE diff(%) Mean,13.983733,18.383543,13.750262,9.039091,12.275015
RMSE diff(%) Stdev,7.639165,6.766103,6.680701,5.674067,7.117026
